# Demo of using Cluster class to manage services and nodes

### Get all necessary import to initialize connection and cluster

In [ ]:
from mstrio.connection import Connection
from mstrio.admin.server import Cluster

base_url = "https://env-225486.customer.cloud.microstrategy.com/MicroStrategyLibrary/api"
username = "mstr"
password = 'LOMEirGHOR65'

# MY DEV
username= "mstr"
password = 'RcitnH9cdKIJ'
base_url ="https://env-231935.customer.cloud.microstrategy.com/MicroStrategyLibrary"

connection = Connection(base_url, username, password, login_mode=1)
cluster = Cluster(connection=connection)

# View nodes and services status on cluster

### List basic info about nodes

In [ ]:
cluster.list_nodes()

In [ ]:
nodes = [node['name'] for node in cluster.list_nodes()]
nodes

### List services grouped by services

In [ ]:
tmp_s = cluster.list_services(group_by='services')
tmp_s

### List services grouped by nodes

In [ ]:
tmp_n = cluster.list_services(group_by='nodes')
tmp_n

### Show topologies of nodes and services (like in Workstation)

In [ ]:
cluster.nodes_topology()

In [ ]:
cluster.services_topology()

# Stop and start chosen service

In [ ]:
def get_service_statuses(cluster, service_name):
    services = cluster.list_services(group_by='services')
    service = [s for s in services if s['service'] == service_name]
    if not service:
        return 'Not service with name' + service_name + '.'
    service = service[0]
    return {'service': service['service'],
            'nodes' : [{n['node'] : n['status']} for n in service['nodes']]}

In [ ]:
get_service_statuses(cluster, 'Apache-Kafka')

In [ ]:
cluster.start(service='Apache-Kafka', nodes=nodes)

In [ ]:
cluster.start(service='Apache-ZooKeeper', nodes=nodes)

In [ ]:
cluster.stop(service='Apache-ZooKeeper', nodes=nodes)

In [ ]:
cluster.stop(service='Apache-Kafka', nodes=nodes)

In [ ]:
get_service_statuses(cluster, 'Apache-Kafka')

In [ ]:
cluster.start(service='Apache-Kafka', nodes=['env-225486laio2use1'])

In [ ]:
get_service_statuses(cluster, 'Apache-Kafka')

# Modify cluster membership and node settings

### Set primary server

In [ ]:
cluster.default_node

In [ ]:
cluster.set_primary_node(name='env-225486laio1use1')

### Remove node from cluster

In [ ]:
cluster.remove_node(name='env-225486laio2use1')

### Add node to cluster

In [ ]:
[n['name'] for n in cluster.list_nodes()]

In [ ]:
cluster.add_node(name='env-225486laio2use1')

In [ ]:
[n['name'] for n in cluster.list_nodes()]

### Change Node Settings

In [ ]:
cluster.list_node_settings('env-225486laio2use1')

In [ ]:
cluster.update_node_settings('env-225486laio2use1', load_balance_factor=2, initial_pool_size=100, max_pool_size=1000)

In [ ]:
cluster.reset_node_settings('env-225486laio2use1')

# Load / Unload Applications

### Unload application from a node

In [ ]:
def get_project_status_on_node(cluster, project_name, node_name):
    return [[{'status': p['status']} 
               for p in n['projects'] if p['name'] == project_name]
             for n in cluster.list_nodes() if n['name'] == node_name][0][0]

In [ ]:
get_project_status_on_node(cluster, 'MicroStrategy Tutorial', 'env-225486laio2use1')

In [ ]:
cluster.unload_application(application_name='MicroStrategy Tutorial',
                           on_nodes='env-225486laio2use1')

### Load application on a node

In [ ]:
cluster.load_application(application_name='MicroStrategy Tutorial',
                         on_nodes='env-225486laio2use1')

# Fetch, view, modify I-Server Settings

In [ ]:
from mstrio.admin.application import Environment

In [ ]:
env = Environment(connection)

In [ ]:
env.server_settings.to_dataframe()

In [ ]:
env.server_settings.hLAutoDeleteTimestamp

In [ ]:
env.server_settings.maxUserConnectionPerServer

## View Detailed Settings information

In [ ]:
env.server_settings.setting_types

## Import/Export Settings

In [ ]:
env.server_settings.to_csv('iserver settings.csv')

In [ ]:
env.server_settings.to_json('iserver settings.json')

In [ ]:
env.server_settings.to_pickle('iserver settings.pickle')

In [ ]:
env.server_settings.import_from('iserver settings.json')

In [ ]:
env.server_settings.import_from('iserver settings.csv')

In [ ]:
env.server_settings.import_from('iserver settings.pickle')

## Push settings to I-Server

In [ ]:
env.server_settings.update()